In [53]:
import pandas as pd
import pg8000
import dateutil.parser
%matplotlib inline

In [38]:
conn = pg8000.connect(host="training.c1erymiua9dx.us-east-1.rds.amazonaws.com", user='dot_student', password='qgis', database='training')
cursor = conn.cursor()

In [39]:
cursor.execute("select * from dot_311")
data = []
for row in cursor.fetchall():
    data.append(row)

In [40]:
#column name 
statement = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'dot_311'"
cursor.execute(statement)
columns = []
for row in cursor.fetchall():
    columns.append(row[0])
print(columns)

['gid', 'unique_key', 'agency', 'agency nam', 'complaint', 'descriptor', 'location t', 'incident z', 'incident a', 'street nam', 'cross stre', 'cross st_1', 'intersecti', 'intersec_1', 'address ty', 'city', 'landmark', 'facility t', 'status', 'due date', 'resolution', 'resoluti_1', 'community', 'borough', 'x coordina', 'y coordina', 'park facil', 'park borou', 'school nam', 'school num', 'school reg', 'school cod', 'school pho', 'school add', 'school cit', 'school sta', 'school zip', 'school not', 'school or', 'vehicle ty', 'taxi compa', 'taxi pick', 'bridge hig', 'bridge h_1', 'road ramp', 'bridge h_2', 'garage lot', 'ferry dire', 'ferry term', 'latitude', 'longitude', 'location', 'geom', 'created_date', 'closed_date']


In [41]:
df=pd.DataFrame(data=data, columns=columns)

In [42]:
df.head()

,gid,unique_key,agency,agency nam,complaint,descriptor,location t,incident z,incident a,street nam,...,bridge h_2,garage lot,ferry dire,ferry term,latitude,longitude,location,geom,created_date,closed_date
0,2,32570549,DOT,Department of Transportation,Traffic Signal Condition,Controller,None,10301,None,None,...,None,None,None,None,40.613803753157917,-74.113373738864198,"(40.61380375315792, -74.1133737388642)",0101000020E61000006829ED83418752C0D7FC121F914E...,2016-02-01 00:12:00,2016-02-01 01:15:00
1,3,32572958,DOT,Department of Transportation,Traffic Signal Condition,Controller,None,10301,None,None,...,None,None,None,None,40.626545297464929,-74.091582334480577,"(40.62654529746493, -74.09158233448058)",0101000020E610000011DF267CDC8552C08809E5A23250...,2016-02-01 00:14:00,2016-02-01 00:54:00
2,4,32573576,DOT,Department of Transportation,Traffic Signal Condition,Controller,None,10310,None,None,...,None,None,None,None,40.629249554787492,-74.123220752604595,"(40.62924955478749, -74.1232207526046)",0101000020E610000004A84BD9E28752C0DC6AD93F8B50...,2016-02-01 00:15:00,2016-02-01 01:30:00
3,5,32572389,DOT,Department of Transportation,Traffic Signal Condition,Controller,None,None,None,None,...,None,None,None,None,None,None,None,0101000020E610000004A84BD9E28752C0DC6AD93F8B50...,2016-02-01 00:17:00,2016-02-01 02:35:00
4,6,32570548,DOT,Department of Transportation,Traffic Signal Condition,Controller,None,None,None,None,...,None,None,None,None,None,None,None,0101000020E610000004A84BD9E28752C0DC6AD93F8B50...,2016-02-01 00:19:00,2016-02-01 01:00:00


In [43]:
new_df=df[['unique_key','agency nam','complaint','created_date','closed_date','status','due date','borough']]

In [44]:
new_df.columns

Index(['unique_key', 'agency nam', 'complaint', 'created_date', 'closed_date',
       'status', 'due date', 'borough'],
      dtype='object')

In [46]:
new_df['new_created_date'] = pd.to_datetime(new_df['created_date'],format='%m/%d/%y %H:%M:%S')

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [47]:
new_df['new_closed_date'] = pd.to_datetime(new_df['closed_date'],format='%m/%d/%y %H:%M:%S')

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [48]:
new_df['time']=new_df['closed_date']-new_df['created_date']

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [49]:
# clean the dirty data
def time_to_day(td):
    try: #not sure all the NaN data types, use try
        return td.days
    except:
        return 0 #turn all the null number into 0
new_df['cleantime']=new_df['time'].apply(time_to_day)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [50]:
new_df['cleantime'].describe()

count    5851.000000
mean        6.021022
std        14.192718
min       -19.000000
25%         0.000000
50%         0.000000
75%         3.000000
max        89.000000
Name: cleantime, dtype: float64

In [73]:
df1 = new_df[new_df['cleantime']>0] #remove the negative cleantime

In [74]:
#standard deviation: average distance of each data point from mean
df1

,unique_key,agency nam,complaint,created_date,closed_date,status,due date,borough,new_created_date,new_closed_date,time,cleantime
11,32570682,Department of Transportation,Street Light Condition,2016-02-01 00:44:00,2016-02-08 12:29:00,Closed,None,BROOKLYN,2016-02-01 00:44:00,2016-02-08 12:29:00,7 days 11:45:00,7
12,32572906,Department of Transportation,Traffic Signal Condition,2016-02-01 00:50:00,2016-02-10 09:10:00,Closed,None,BROOKLYN,2016-02-01 00:50:00,2016-02-10 09:10:00,9 days 08:20:00,9
16,32570317,Department of Transportation,Street Condition,2016-02-01 01:18:49,2016-02-03 16:59:00,Closed,03/02/2016 01:18:49 AM,BROOKLYN,2016-02-01 01:18:49,2016-02-03 16:59:00,2 days 15:40:11,2
17,32581124,Department of Transportation,Street Light Condition,2016-02-01 01:20:00,2016-02-05 12:04:00,Closed,None,QUEENS,2016-02-01 01:20:00,2016-02-05 12:04:00,4 days 10:44:00,4
18,32579277,Department of Transportation,Street Light Condition,2016-02-01 01:36:00,2016-02-04 10:37:00,Closed,None,BRONX,2016-02-01 01:36:00,2016-02-04 10:37:00,3 days 09:01:00,3
19,32580720,Department of Transportation,Street Light Condition,2016-02-01 01:36:00,2016-02-02 13:09:00,Closed,None,STATEN ISLAND,2016-02-01 01:36:00,2016-02-02 13:09:00,1 days 11:33:00,1
21,32580285,Department of Transportation,Street Light Condition,2016-02-01 02:07:00,2016-02-08 15:25:00,Closed,None,QUEENS,2016-02-01 02:07:00,2016-02-08 15:25:00,7 days 13:18:00,7
22,32575419,Department of Transportation,Sidewalk Condition,2016-02-01 02:32:08,2016-02-04 12:06:14,Closed,04/01/2016 02:32:08 AM,BROOKLYN,2016-02-01 02:32:08,2016-02-04 12:06:14,3 days 09:34:06,3
24,32581170,Department of Transportation,Street Light Condition,2016-02-01 03:21:00,2016-02-02 14:57:00,Closed,None,BROOKLYN,2016-02-01 03:21:00,2016-02-02 14:57:00,1 days 11:36:00,1
27,32577440,Department of Transportation,Street Light Condition,2016-02-01 03:26:00,2016-02-03 10:03:00,Closed,None,MANHATTAN,2016-02-01 03:26:00,2016-02-03 10:03:00,2 days 06:37:00,2


In [52]:
#range
df1['cleantime'].max()-df1['cleantime'].min()

108

In [79]:
#Interquartile Range: middle 50% of the data
iqr=df1['cleantime'].quantile(q=0.75)-df1['cleantime'].quantile(q=0.25)
iqr

27.0

In [81]:
#ual
ual = df1['cleantime'].quantile(q=0.75) + (iqr* 1.5)
ual

68.5

In [84]:
#lal
lal=df1['cleantime'].quantile(q=0.25) - (iqr*1.5) 
lal

-39.5

In [87]:
#outlier
outlier = df1[df1['cleantime']>ual]

In [88]:
outlier.describe()

,time,cleantime
count,33,33.000000
mean,78 days 14:22:40.303030,78.030303
std,6 days 14:42:00.990269,6.659039
min,69 days 00:41:00,69.000000
25%,71 days 23:58:00,71.000000
50%,78 days 18:49:00,78.000000
75%,84 days 03:08:00,84.000000
max,89 days 18:54:00,89.000000
